# Multiprocessing in Python

## Multiprocessing with Multiprocessing library

In [9]:
import multiprocessing
import time


def cpu_bound(number):
    # print(multiprocessing.current_process().name)
    return sum(i * i for i in range(number))


def find_sums(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(cpu_bound, numbers)


In [10]:
numbers = [5_000_000 + x for x in range(20)]

start_time = time.perf_counter()
find_sums(numbers)
duration = time.perf_counter() - start_time
print(f"Duration {duration} seconds")

Duration 9.50056497901096 seconds


## Multiprocessing with concurrent.futures library

In [11]:
from concurrent.futures import ProcessPoolExecutor

In [12]:
start_time = time.perf_counter()
with ProcessPoolExecutor() as executor:
    executor.map(cpu_bound, numbers)
print(f"Duration {time.perf_counter()-start_time} seconds")

Duration 9.688510706997477 seconds


## Multiprocessing for Pandas Apply

### With Multiprocessing Standard library

In [5]:
from typing import Union, Callable
import multiprocessing as mp
import numpy as np
import pandas as pd
from tqdm import tqdm

def parallel_apply(
        df_or_s: Union[pd.DataFrame, pd.Series],
        func: Callable,
        desc: str,
        n_jobs: int = mp.cpu_count()-1,
        progress_bar: bool = True
    ) -> Union[pd.DataFrame, pd.Series]:
    """A method to use multiprocessing for `df.apply()` or `s.apply()`.

    Args:
        df_or_s (Union[pd.DataFrame, pd.Series]): Dataframe or series to
        use `df.apply()` or `s.apply()` on.
        func (Callable): The apply function to be used in
        `df.apply()` or `s.apply()`.
        n_jobs (int, optional): Number of processors to use.
        Defaults to mp.cpu_count()-1.
        progress_bar (bool, optional): A tqdm progress bar.
        Defaults to True.
        desc (str): TQDM description kwarg.

    Returns:
        Union[pd.DataFrame, pd.Series]: Returns the original dataframe or
        series after applying the `df.apply()` or `s.apply()`.
    """
    with mp.Pool(n_jobs) as pool:
        split = np.array_split(df_or_s, n_jobs * 2)
        if progress_bar is True:
            split = tqdm(split, desc=desc)
        ret_list = pool.map(func, split)
        output_df_or_s = pd.concat(ret_list)
    
    return output_df_or_s


### With Joblib library

In [6]:
from typing import Union, Callable
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

def parallel_apply(
        df_or_s: Union[pd.DataFrame, pd.Series],
        func: Callable,
        desc: str,
        n_jobs: int = joblib.cpu_count()-1,
        progress_bar: bool = True
    ) -> Union[pd.DataFrame, pd.Series]:
    """A method to use multiprocessing for `df.apply()` or `s.apply()`.

    Args:
        df_or_s (Union[pd.DataFrame, pd.Series]): Dataframe or series to
        use `df.apply()` or `s.apply()` on.
        func (Callable): The apply function to be used in
        `df.apply()` or `s.apply()`.
        n_jobs (int, optional): Number of processors to use.
        Defaults to mp.cpu_count()-1.
        progress_bar (bool, optional): A tqdm progress bar.
        Defaults to True.
        desc (str): TQDM description kwarg.

    Returns:
        Union[pd.DataFrame, pd.Series]: Returns the original dataframe or
        series after applying the `df.apply()` or `s.apply()`.
    """

    with Parallel(n_jobs=n_jobs, mmap_mode="r+") as parallel:
        split = np.array_split(df_or_s, 2 * n_jobs)
        if progress_bar is True:
            ret_list = parallel(
                delayed(func)(x) for x in tqdm(split, desc=desc))
        else:
            ret_list = parallel(delayed(func)(x) for x in split)
        output_df_or_s = pd.concat(ret_list)

    return output_df_or_s